In [ ]:
import json
from collections import deque, defaultdict
from Models.cube_model_positions_only import Cube
import copy
import numpy as np

In [7]:
def generate_move_table():
    """
    Generates a table mapping each move to the positions it affects and their new positions.

    Returns:
        dict: A dictionary where keys are move names and values are dictionaries
              mapping initial positions to final positions after the move.
    """
    cube = Cube()  # Create a fresh cube in solved_cube state
    movement_table = {}

    # Positions that are not the center and thus their pieces' positions are tracked
    tracked_positions = [(i,j,k) for i in range(0,3) for j in range(0,3) for k in range(0,3)]

    # For each possible move
    for move in cube.move_map.keys():
        # Get fresh cube for each move calculation
        test_cube = Cube()

        # Record initial piece IDs at each tracked position
        initial_piece_ids = {}
        for pos in tracked_positions:
            initial_piece_ids[pos] = test_cube._get_piece_at_position(pos)

        # Apply the move
        test_cube.apply_moves(move)

        # Record final positions based on moved piece IDs
        movements = {}
        for initial_pos in tracked_positions:
            piece_id_to_track = initial_piece_ids[initial_pos]
            final_pos = test_cube._get_position_of_piece(piece_id_to_track) # Find where that piece ended up
            if initial_pos != final_pos:
                movements[initial_pos] = final_pos

        # Store in table
        movement_table[move] = movements

    # Save to file
    with open('position_movement_table.json', 'w') as f:
        # Convert tuple positions to strings for JSON serialization
        serializable_table = {}
        for move, position_movements in movement_table.items():
            serializable_movements = {}
            for from_pos, to_pos in position_movements.items():
                from_pos_str = ','.join(map(str, from_pos))
                to_pos_str = ','.join(map(str, to_pos))
                serializable_movements[from_pos_str] = to_pos_str
            serializable_table[move] = serializable_movements

        json.dump(serializable_table, f, indent=2)

In [8]:
generate_move_table()

In [9]:
def calculate_distance_table(piece_type, filename:str):
    """
    Builds a position graph for the piece type, then BFSes in that position graph
    from the start position to the target position. This ignores the rest of the puzzle.
    The distance table is keyed by positions, not piece IDs.
    """
    solved_cube = Cube()
    all_moves = list(solved_cube.move_map.keys())
    core_moves = copy.deepcopy(all_moves)
    for move in ['M', 'E', 'S', 'm', 'e', 's']:
        core_moves.remove(move)
    valid_positions = solved_cube.edge_positions if piece_type == "edge" else solved_cube.corner_positions

    #load the position_movement_table created in the previous cell:
    try:
        filename_1 = "position_movement_table.json"
        with open(filename_1, 'r') as f:
            serializable_table = json.load(f)
    except Exception as e:
        print(f"Failed to load {filename_1}: {e}")

    pos_graph_to_be_modified = {} # current signature: {move: {position: new_position}}. Required: {position: {move: new_position}}
    for move, position_movements in serializable_table.items():
        movements = {}
        for from_pos_str, to_pos_str in position_movements.items():
            from_pos = tuple(eval(from_pos_str))
            to_pos = tuple(eval(to_pos_str))
            movements[from_pos] = to_pos
        pos_graph_to_be_modified[move] = movements

    # create new pos_graph:
    pos_graph = defaultdict(dict)

    # new pos_graph generator:
    for move, position_changes in pos_graph_to_be_modified.items():
        for init_position, final_position in position_changes.items():
            pos_graph[init_position][move] = final_position

    distance_table = {}
    for start_pos in valid_positions: # Iterate over positions, not piece_ids
        for target_pos in valid_positions:
            if start_pos == target_pos:
                distance_table[(start_pos, target_pos)] = 0
                continue

            if (target_pos, start_pos) in distance_table: #symmetry optimization
                distance_table[(start_pos, target_pos)] = distance_table[(target_pos, start_pos)]
                continue

            # BFS in the position graph from start_pos to target_pos
            visited = set([start_pos])
            queue = deque([(start_pos, 0)])
            found_distance = -1

            while queue:
                current_pos, dist = queue.popleft()
                if current_pos == target_pos:
                    found_distance = dist
                    break
                # Explore all possible moves from current_pos in the graph
                for move in pos_graph[current_pos]:
                    if move not in core_moves:
                        continue
                    else:
                        next_pos = pos_graph[current_pos][move]
                        if next_pos not in visited:
                            visited.add(next_pos)
                            queue.append((next_pos, dist + 1))

            distance_table[(start_pos, target_pos)] = found_distance

    serializable_table = {}
    for pos_pair, dist in distance_table.items(): # Key is now position pair
        serializable_table[str(pos_pair)] = dist
    with open(filename, 'w') as f:
        json.dump(serializable_table, f, indent=2)

In [10]:
calculate_distance_table("edge", "edge_position_distance_table.json")
calculate_distance_table("corner", "corner_position_distance_table.json")